# All PPTs

In [97]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [98]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [99]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 745 kB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [100]:
!pip install pyLDAvis==3.2.1

In [101]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [102]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [103]:
all = pd.read_csv('User Needs Assessments Data.csv')
all = all.loc[all['Person'] != 'Interviewer']

In [104]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Question/Answer']))

print(data_words[:1])

[['bought', 'the', 'place', 'years', 'ago', 'but', 'it', 'still', 'under', 'renovation']]


In [105]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['bought', 'the', 'place', 'years', 'ago', 'but', 'it', 'still', 'under', 'renovation']


In [106]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [107]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['buy', 'place', 'year', 'ago', 'still', 'renovation']]


In [108]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [109]:
id2word[0]

'ago'

In [110]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ago', 1),
  ('buy', 1),
  ('place', 1),
  ('renovation', 1),
  ('still', 1),
  ('year', 1)]]

In [111]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [112]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.100*"know" + 0.072*"go" + 0.053*"think" + 0.045*"online" + '
  '0.043*"service" + 0.036*"maybe" + 0.035*"mobile" + 0.033*"mean" + '
  '0.033*"thing" + 0.026*"example"'),
 (1,
  '0.071*"result" + 0.053*"come" + 0.029*"actually" + 0.025*"inaudible" + '
  '0.021*"inform" + 0.021*"term" + 0.020*"call" + 0.016*"website" + '
  '0.015*"know" + 0.015*"today"'),
 (2,
  '0.130*"person" + 0.090*"change" + 0.087*"really" + 0.080*"apply" + '
  '0.075*"department" + 0.065*"actually" + 0.063*"go" + 0.062*"even" + '
  '0.043*"question" + 0.041*"register"'),
 (3,
  '0.152*"laugh" + 0.127*"receive" + 0.112*"much" + 0.035*"term" + 0.030*"let" '
  '+ 0.024*"site" + 0.018*"different" + 0.018*"whatnot" + 0.013*"electricity" '
  '+ 0.013*"water"'),
 (4,
  '0.084*"sale" + 0.026*"eco" + 0.026*"contribution" + 0.026*"visit" + '
  '0.023*"half" + 0.017*"ehm" + 0.017*"band" + 0.017*"unit" + '
  '0.017*"government" + 0.017*"usually"'),
 (5,
  '0.077*"arm" + 0.076*"accompany" + 0.045*"provide" + 0.031*"qu

In [113]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.423005380407647

Coherence Score:  0.3497337851419796


In [114]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.137631  0.347706       1        1  15.119838
14     0.100205 -0.354010       2        1  13.893700
19     0.248115  0.300878       3        1  12.500020
15    -0.352370 -0.107703       4        1   9.978970
13    -0.238239  0.179236       5        1   5.955539
12     0.066618  0.335792       6        1   5.767135
8      0.328923  0.012875       7        1   5.397149
17    -0.325948  0.049368       8        1   5.233883
1      0.037140  0.096501       9        1   3.803851
3     -0.153552 -0.137884      10        1   3.292680
2     -0.132248 -0.329347      11        1   3.234930
6      0.097498 -0.128153      12        1   3.196073
11    -0.138313  0.076174      13        1   3.083172
5      0.193860 -0.144476      14        1   3.071534
4      0.173842 -0.046303      15        1   2.652264
16     0.038119 -0.003425      16        1   1.237264
18     0.115759  0.047418      17        1   0.933200
7      0.010303 -0.115027      18        1   0.823751
10     0.028263 -0.047825      19        1   0.714951
9      0.039654 -0.031796      20        1   0.110096, topic_info=          Term       Freq      Total Category  logprob  loglift
28     service  37.000000  37.000000  Default  30.0000  30.0000
114      think  47.000000  47.000000  Default  29.0000  29.0000
26        form  43.000000  43.000000  Default  28.0000  28.0000
138       mean  19.000000  19.000000  Default  27.0000  27.0000
8          pay  35.000000  35.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
114      think   0.003957  47.440172  Topic20  -6.3206  -2.5801
228  inaudible   0.003957   4.896420  Topic20  -6.3206  -0.3092
434       fair   0.003957   1.160831  Topic20  -6.3206   1.1302
435     little   0.003957   1.160831  Topic20  -6.3206   1.1302
35       first   0.003957   4.009786  Topic20  -6.3206  -0.1094

[874 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
203      14  0.846797     access
428      14  0.935688  accompany
121       2  0.320687    account
121       4  0.320687    account
121       6  0.256549    account
...     ...       ...        ...
92        3  0.915522       work
48        2  0.462801      wrong
5        14  0.325457       year
5        19  0.325457       year
399      14  0.846819        yet

[473 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 15, 20, 16, 14, 13, 9, 18, 2, 4, 3, 7, 12, 6, 5, 17, 19, 8, 11, 10])

In [115]:
pyLDAvis.save_html(vis, 'All PPTs.html')

# PPT01

In [116]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT01 = all.loc[(all['Person'] == 'PPT01') & (all['Person'] != 'Interviewer')]

In [117]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT01['Question/Answer']))

print(data_words[:1])

[['bought', 'the', 'place', 'years', 'ago', 'but', 'it', 'still', 'under', 'renovation']]


In [118]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['bought', 'the', 'place', 'years', 'ago', 'but', 'it', 'still', 'under', 'renovation']


In [119]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [120]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['buy', 'place', 'year', 'ago', 'still', 'renovation']]


In [121]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [122]:
id2word[0]

'ago'

In [123]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ago', 1),
  ('buy', 1),
  ('place', 1),
  ('renovation', 1),
  ('still', 1),
  ('year', 1)]]

In [124]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [125]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"customer" + 0.039*"website" + 0.039*"people" + 0.039*"form" + '
  '0.039*"ideally" + 0.027*"fill" + 0.026*"touchpoint" + 0.026*"available" + '
  '0.026*"need" + 0.026*"help"'),
 (1,
  '0.049*"additional" + 0.049*"document" + 0.049*"card" + 0.049*"number" + '
  '0.049*"send" + 0.049*"time" + 0.049*"d" + 0.049*"monthly" + 0.049*"thing" + '
  '0.049*"basis"'),
 (2,
  '0.089*"mobile" + 0.060*"app" + 0.060*"website" + 0.031*"operate" + '
  '0.031*"factor" + 0.031*"know" + 0.031*"convert" + 0.031*"least" + '
  '0.031*"information" + 0.031*"support"'),
 (3,
  '0.057*"form" + 0.044*"service" + 0.044*"payment" + 0.032*"pay" + '
  '0.026*"take" + 0.026*"go" + 0.026*"time" + 0.019*"tell" + 0.019*"reply" + '
  '0.019*"usually"'),
 (4,
  '0.004*"along" + 0.004*"agent" + 0.004*"bubble" + 0.004*"bot" + '
  '0.004*"available" + 0.004*"amount" + 0.004*"clearly" + 0.004*"science" + '
  '0.004*"want" + 0.004*"close"'),
 (5,
  '0.042*"pay" + 0.041*"utility" + 0.041*"service" + 0.041*"bill" 

In [126]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.9172383365722805

Coherence Score:  0.5357074207249144


In [127]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.133078  0.201856       1        1  27.957606
0      0.098392 -0.269226       2        1  12.726336
16    -0.024841  0.259803       3        1  10.506665
12    -0.098565 -0.261275       4        1   8.216040
15    -0.222227  0.137219       5        1   7.167125
13     0.230334  0.176128       6        1   7.051200
19     0.181474 -0.099010       7        1   5.045361
10    -0.194503  0.034944       8        1   4.457601
2      0.249627  0.029865       9        1   4.454570
9     -0.183870 -0.090778      10        1   3.595670
5     -0.092120 -0.080081      11        1   2.729867
6     -0.001422 -0.124146      12        1   2.150132
1      0.044746  0.111873      13        1   1.874453
8      0.070166 -0.030933      14        1   0.936312
18     0.030785  0.016651      15        1   0.443262
7      0.009003 -0.002735      16        1   0.143354
14     0.009025 -0.002539      17        1   0.136195
11     0.009025 -0.002539      18        1   0.136084
4      0.009025 -0.002539      19        1   0.136084
17     0.009025 -0.002539      20        1   0.136084, topic_info=                Term       Freq      Total Category  logprob  loglift
8                pay  15.000000  15.000000  Default   30.000  30.0000
13              time   6.000000   6.000000  Default   29.000  29.0000
26              form  18.000000  18.000000  Default   28.000  28.0000
28           service  13.000000  13.000000  Default   27.000  27.0000
176          website   7.000000   7.000000  Default   26.000  26.0000
..               ...        ...        ...      ...      ...      ...
25              fine   0.003117   2.235228  Topic20   -5.425   0.0245
26              form   0.003117  18.330788  Topic20   -5.425  -2.0797
27           payment   0.003117   9.093897  Topic20   -5.425  -1.3787
28           service   0.003117  13.711989  Topic20   -5.425  -1.7894
29   straightforward   0.003117   1.317544  Topic20   -5.425   0.5531

[937 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
203       6  0.801140    access
121       3  0.759252   account
208       4  0.784584    actual
148       2  0.739398  actually
115       1  0.343555       add
...     ...       ...       ...
92        1  0.456112      work
92        4  0.456112      work
48        1  0.461783     wrong
48        5  0.461783     wrong
5         4  0.784505      year

[357 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 17, 13, 16, 14, 20, 11, 3, 10, 6, 7, 2, 9, 19, 8, 15, 12, 5, 18])

In [128]:
pyLDAvis.save_html(vis, 'PPT01.html')

# PPT02

In [129]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT02 = all.loc[(all['Person'] == 'PPT02') & (all['Person'] != 'Interviewer')]

In [130]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT02['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']]


In [131]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']


In [132]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [133]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'bill', 'get', 'estimate', 'upcoming', 'bill', 'put', 'inaudible', 'meter', 'reading', 'get', 'estimate', 'monthly', 'bill', 'check', 'interruption', 'service']]


In [134]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [135]:
id2word[0]

'bill'

In [136]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bill', 3),
  ('check', 1),
  ('estimate', 2),
  ('get', 2),
  ('inaudible', 1),
  ('interruption', 1),
  ('meter', 1),
  ('monthly', 1),
  ('pay', 1),
  ('put', 1),
  ('reading', 1),
  ('service', 1),
  ('upcoming', 1)]]

In [137]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [138]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.005*"least" + 0.005*"phonecall" + 0.005*"acknowledge" + 0.005*"come" + '
  '0.005*"expect" + 0.005*"good" + 0.005*"government" + 0.005*"happen" + '
  '0.005*"inform" + 0.005*"installation"'),
 (1,
  '0.052*"mean" + 0.052*"form" + 0.052*"fill" + 0.052*"meter" + 0.027*"call" + '
  '0.027*"anyway" + 0.027*"time" + 0.027*"inaudible" + 0.027*"install" + '
  '0.027*"expect"'),
 (2,
  '0.006*"customer" + 0.006*"account" + 0.006*"support" + 0.006*"mean" + '
  '0.006*"advise" + 0.006*"know" + 0.006*"figure" + 0.006*"quite" + '
  '0.006*"number" + 0.006*"care"'),
 (3,
  '0.005*"least" + 0.005*"phonecall" + 0.005*"acknowledge" + 0.005*"come" + '
  '0.005*"expect" + 0.005*"good" + 0.005*"government" + 0.005*"happen" + '
  '0.005*"inform" + 0.005*"installation"'),
 (4,
  '0.049*"customer" + 0.037*"account" + 0.025*"really" + 0.025*"say" + '
  '0.025*"tell" + 0.025*"call" + 0.025*"advise" + 0.025*"figure" + '
  '0.025*"care" + 0.025*"login"'),
 (5,
  '0.080*"service" + 0.048*"mobile" + 0.0

In [139]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.710982562560174

Coherence Score:  0.7217597410864645


In [140]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.121293  0.226324       1        1  16.790627
12     0.041610 -0.248330       2        1  14.908448
5     -0.244977 -0.127102       3        1  12.488123
11    -0.153277  0.226337       4        1  12.411077
10     0.147990 -0.179595       5        1  10.178876
19     0.257571 -0.086029       6        1   8.095361
1     -0.016134  0.216462       7        1   6.728787
15    -0.137476 -0.218677       8        1   5.406602
9     -0.214372  0.073908       9        1   4.620200
8      0.169627  0.119920      10        1   4.520337
13     0.150224 -0.000720      11        1   2.775781
18    -0.013898 -0.001285      12        1   0.129688
14    -0.013046 -0.000357      13        1   0.121643
2     -0.013491  0.000141      14        1   0.121595
6     -0.013607 -0.000166      15        1   0.117142
7     -0.013608 -0.000166      16        1   0.117142
3     -0.013607 -0.000166      17        1   0.117142
16    -0.013608 -0.000166      18        1   0.117142
17    -0.013607 -0.000166      19        1   0.117142
0     -0.013607 -0.000166      20        1   0.117142, topic_info=        Term       Freq      Total Category  logprob  loglift
11   service  12.000000  12.000000  Default   30.000  30.0000
28      form  11.000000  11.000000  Default   29.000  29.0000
14     think   7.000000   7.000000  Default   28.000  28.0000
0       bill   5.000000   5.000000  Default   27.000  27.0000
27      fill   7.000000   7.000000  Default   26.000  26.0000
..       ...        ...        ...      ...      ...      ...
23  customer   0.002781   6.317735  Topic20   -5.204  -0.9789
24  download   0.002781   2.824002  Topic20   -5.204  -0.1737
26     email   0.002781   2.858916  Topic20   -5.204  -0.1860
27      fill   0.002781   7.797787  Topic20   -5.204  -1.1894
28      form   0.002781  11.930906  Topic20   -5.204  -1.6147

[992 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
143       2  0.763866         able
51        1  0.477134      account
51        4  0.477134      account
51        9  0.159045      account
117       5  0.784462  acknowledge
...     ...       ...          ...
134       5  0.784441         week
44        1  0.259952         well
44        2  0.519904         well
44        6  0.259952         well
160       2  0.763870         year

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 13, 6, 12, 11, 20, 2, 16, 10, 9, 14, 19, 15, 3, 7, 8, 4, 17, 18, 1])

In [141]:
pyLDAvis.save_html(vis, 'PPT02.html')

# PPT03

In [142]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT03 = all.loc[(all['Person'] == 'PPT03') & (all['Person'] != 'Interviewer')]

In [143]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT03['Question/Answer']))

print(data_words[:1])

[['laughs', 'because', 'it', 'the', 'only', 'place', 'to', 'do', 'so', 'no', 'in', 'malta', 'no', 'there', 'no', 'other', 'form', 'of', 'receiving', 'water', 'and', 'electricity', 'through', 'different', 'ehm', 'correct', 'me', 'if', 'wrong', 'but', 'it', 'the', 'only', 'solution', 'to', 'doing', 'so']]


In [144]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['laughs', 'because', 'it', 'the', 'only', 'place', 'to', 'do', 'so', 'no', 'in', 'malta', 'no', 'there', 'no', 'other', 'form', 'of', 'receiving', 'water', 'and', 'electricity', 'through', 'different', 'ehm', 'correct', 'me', 'if', 'wrong', 'but', 'it', 'the', 'only', 'solution', 'to', 'doing', 'so']


In [145]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [146]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['laugh', 'place', 'form', 'receive', 'water', 'electricity', 'different', 'ehm', 'correct', 'wrong', 'solution']]


In [147]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [148]:
id2word[0]

'correct'

In [149]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('correct', 1),
  ('different', 1),
  ('ehm', 1),
  ('electricity', 1),
  ('form', 1),
  ('laugh', 1),
  ('place', 1),
  ('receive', 1),
  ('solution', 1),
  ('water', 1),
  ('wrong', 1)]]

In [150]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [151]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"experience" + 0.006*"much" + 0.006*"typical" + 0.006*"ui" + '
  '0.006*"user" + 0.006*"whatnot" + 0.006*"abroad" + 0.006*"apply" + '
  '0.006*"argue" + 0.006*"site"'),
 (1,
  '0.082*"online" + 0.042*"easily" + 0.042*"attach" + 0.042*"send" + '
  '0.042*"laugh" + 0.042*"document" + 0.042*"form" + 0.042*"system" + '
  '0.021*"stupid" + 0.021*"first"'),
 (2,
  '0.073*"term" + 0.073*"form" + 0.037*"good" + 0.037*"easy" + 0.037*"come" + '
  '0.037*"example" + 0.037*"manually" + 0.037*"make" + 0.037*"experience" + '
  '0.037*"moment"'),
 (3,
  '0.007*"quite" + 0.007*"think" + 0.007*"easy" + 0.007*"straightforward" + '
  '0.006*"read" + 0.006*"make" + 0.006*"yet" + 0.006*"browse" + 0.006*"d" + '
  '0.006*"enjoyable"'),
 (4,
  '0.073*"account" + 0.055*"change" + 0.037*"proof" + 0.037*"order" + '
  '0.037*"call" + 0.019*"really" + 0.019*"laugh" + 0.019*"low" + '
  '0.019*"always" + 0.019*"expectation"'),
 (5,
  '0.087*"bill" + 0.086*"receive" + 0.058*"issue" + 0.058*"laugh" + '
 

In [152]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.730853134467278

Coherence Score:  0.6579006829412793


In [153]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.260415 -0.017613       1        1  14.353291
1      0.234691  0.174615       2        1  12.355950
10    -0.233569  0.145946       3        1   8.616336
14    -0.265345 -0.015640       4        1   8.616186
5      0.106532 -0.258081       5        1   8.314767
9     -0.098658 -0.227430       6        1   8.307939
16    -0.200160 -0.124369       7        1   7.388423
19    -0.065221 -0.187790       8        1   6.457051
2      0.007546  0.243340       9        1   6.148312
17     0.122393 -0.144722      10        1   4.305076
18     0.163324 -0.047310      11        1   4.298874
12    -0.053426  0.178695      12        1   4.174765
6      0.103700  0.167309      13        1   3.997432
11    -0.083889  0.049918      14        1   1.844714
7      0.001240  0.010569      15        1   0.140914
3     -0.000436  0.010834      16        1   0.140744
8      0.000215  0.010432      17        1   0.134807
13     0.000215  0.010432      18        1   0.134807
15     0.000216  0.010432      19        1   0.134807
0      0.000215  0.010432      20        1   0.134807, topic_info=         Term      Freq     Total Category  logprob  loglift
89     really  7.000000  7.000000  Default  30.0000  30.0000
4        form  5.000000  5.000000  Default  29.0000  29.0000
14     online  4.000000  4.000000  Default  28.0000  28.0000
11       bill  5.000000  5.000000  Default  27.0000  27.0000
34       take  5.000000  5.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
25  initially  0.002663  2.121618  Topic20  -5.0999  -0.0713
26       last  0.002663  1.280349  Topic20  -5.0999   0.4337
27    lengthy  0.002663  1.996790  Topic20  -5.0999  -0.0107
28        let  0.002663  4.249163  Topic20  -5.0999  -0.7658
29       mean  0.002663  2.541206  Topic20  -5.0999  -0.2518

[935 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
110       6  0.786999    abroad
17        1  0.645776   account
17        4  0.215259   account
18        4  0.781043  actually
38        1  0.742762   already
...     ...       ...       ...
109       8  0.286271   whatnot
147       5  0.788191      work
10       12  0.896296     wrong
37        4  0.781041      year
160       3  0.783299       yet

[268 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 11, 15, 6, 10, 17, 20, 3, 18, 19, 13, 7, 12, 8, 4, 9, 14, 16, 1])

In [154]:
pyLDAvis.save_html(vis, 'PPT03.html')

# PPT04

In [155]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT04 = all.loc[(all['Person'] == 'PPT04') & (all['Person'] != 'Interviewer')]

In [156]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT04['Question/Answer']))

print(data_words[:1])

[['ehm', 'bought', 'the', 'place', 'almost', 'two', 'and', 'half', 'years', 'but', 've', 'been', 'living', 'here', 'for', 'year']]


In [157]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['ehm', 'bought', 'the', 'place', 'almost', 'two', 'and', 'half', 'years', 'but', 've', 'been', 'living', 'here', 'for', 'year']


In [158]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [159]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['buy', 'place', 'almost', 'half', 'year', 'live', 'year']]


In [160]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2)]]


In [161]:
id2word[0]

'almost'

In [162]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('almost', 1),
  ('buy', 1),
  ('half', 1),
  ('live', 1),
  ('place', 1),
  ('year', 2)]]

In [163]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [164]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.076*"website" + 0.076*"tell" + 0.076*"payment" + 0.076*"pay" + '
  '0.076*"bill" + 0.076*"truth" + 0.076*"mind" + 0.004*"band" + 0.004*"far" + '
  '0.004*"last"'),
 (1,
  '0.056*"form" + 0.056*"household" + 0.056*"download" + 0.029*"lease" + '
  '0.029*"i" + 0.029*"place" + 0.029*"course" + 0.029*"card" + 0.029*"d" + '
  '0.029*"state"'),
 (2,
  '0.118*"water" + 0.118*"electricity" + 0.079*"service" + 0.041*"bill" + '
  '0.041*"pay" + 0.041*"well" + 0.041*"somewhere" + 0.041*"put" + '
  '0.041*"money" + 0.041*"like"'),
 (3,
  '0.082*"trouble" + 0.082*"maybe" + 0.082*"bill" + 0.082*"time" + '
  '0.082*"know" + 0.082*"indeed" + 0.004*"get" + 0.004*"right" + 0.004*"far" + '
  '0.004*"last"'),
 (4,
  '0.007*"bit" + 0.007*"do" + 0.007*"right" + 0.007*"mind" + 0.007*"far" + '
  '0.007*"issue" + 0.007*"band" + 0.007*"government" + 0.007*"contribution" + '
  '0.007*"example"'),
 (5,
  '0.083*"actually" + 0.042*"arm" + 0.042*"form" + 0.042*"ok" + 0.042*"google" '
  '+ 0.042*"exactly" 

In [165]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.635885872485789

Coherence Score:  0.49915224130107017


In [166]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.236060 -0.092732       1        1  17.626477
19     0.188785  0.225561       2        1  13.864080
1     -0.230935  0.129916       3        1  12.549223
18    -0.268243  0.003107       4        1   9.561044
6      0.149658 -0.226139       5        1   9.142416
2     -0.026946 -0.270957       6        1   7.733000
5     -0.196805 -0.127929       7        1   7.445152
8      0.220221  0.090171       8        1   6.389859
12    -0.030072  0.231240       9        1   4.603786
13    -0.073624 -0.128713      10        1   2.999752
0      0.112489 -0.031383      11        1   2.871681
3     -0.028516  0.126049      12        1   2.451866
15     0.006792  0.045387      13        1   0.925625
10    -0.008894  0.003936      14        1   0.269818
11    -0.008328  0.003748      15        1   0.261037
9     -0.008328  0.003748      16        1   0.261037
7     -0.008328  0.003748      17        1   0.261037
14    -0.008328  0.003748      18        1   0.261037
16    -0.008328  0.003748      19        1   0.261037
4     -0.008328  0.003748      20        1   0.261037, topic_info=       Term      Freq     Total Category  logprob  loglift
36     form  4.000000  4.000000  Default  30.0000  30.0000
10     bill  4.000000  4.000000  Default  29.0000  29.0000
15  website  3.000000  3.000000  Default  28.0000  28.0000
16     know  4.000000  4.000000  Default  27.0000  27.0000
76    visit  2.000000  2.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
25    house  0.004544  1.198577  Topic20  -4.9053   0.3732
26   instal  0.004544  1.198577  Topic20  -4.9053   0.3732
27    meter  0.004544  1.945623  Topic20  -4.9053  -0.1113
28     need  0.004544  2.048412  Topic20  -4.9053  -0.1628
29      old  0.004544  1.198577  Topic20  -4.9053   0.3732

[986 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
81        1  0.768430  accompany
20        1  0.454492    account
20        4  0.227246    account
20        5  0.454492    account
68        7  0.404269   actually
...     ...       ...        ...
15        7  0.287202    website
15        9  0.574404    website
15       11  0.287202    website
133       6  0.863518       well
5         4  1.019356       year

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 20, 2, 19, 7, 3, 6, 9, 13, 14, 1, 4, 16, 11, 12, 10, 8, 15, 17, 5])

In [167]:
pyLDAvis.save_html(vis, 'PPT04.html')

# PPT05

In [168]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT05 = all.loc[(all['Person'] == 'PPT05') & (all['Person'] != 'Interviewer')]

In [169]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT05['Question/Answer']))

print(data_words[:1])

[['ve', 'been', 'homeowner', 'for', 'the', 'past', 'couple', 'of', 'years', 'think', 'three', 'years']]


In [170]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['ve', 'been', 'homeowner', 'for', 'the', 'past', 'couple', 'of', 'years', 'think', 'three', 'years']


In [171]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [172]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['homeowner', 'couple', 'year', 'think', 'year']]


In [173]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2)]]


In [174]:
id2word[0]

'couple'

In [175]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('couple', 1), ('homeowner', 1), ('think', 1), ('year', 2)]]

In [176]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [177]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.062*"website" + 0.050*"bill" + 0.037*"honest" + 0.037*"pay" + '
  '0.037*"know" + 0.025*"think" + 0.025*"avoid" + 0.025*"message" + '
  '0.025*"mobile" + 0.025*"start"'),
 (1,
  '0.120*"form" + 0.081*"know" + 0.081*"fill" + 0.041*"go" + 0.041*"user" + '
  '0.041*"letter" + 0.041*"sure" + 0.041*"remember" + 0.041*"new" + '
  '0.041*"person"'),
 (2,
  '0.004*"mean" + 0.004*"internet" + 0.004*"old" + 0.004*"nation" + '
  '0.004*"move" + 0.004*"mentality" + 0.004*"overseas" + 0.004*"extent" + '
  '0.004*"forward" + 0.004*"regard"'),
 (3,
  '0.091*"go" + 0.047*"website" + 0.047*"person" + 0.047*"register" + '
  '0.047*"honest" + 0.047*"department" + 0.047*"completely" + 0.047*"apply" + '
  '0.047*"even" + 0.047*"actually"'),
 (4,
  '0.083*"eventually" + 0.005*"know" + 0.005*"sort" + 0.005*"person" + '
  '0.005*"way" + 0.005*"stuff" + 0.005*"service" + 0.005*"go" + 0.004*"even" + '
  '0.004*"mean"'),
 (5,
  '0.004*"mean" + 0.004*"internet" + 0.004*"old" + 0.004*"nation" + '
  '0.00

In [178]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.950132364242719

Coherence Score:  0.5817905302483501


In [179]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.214721 -0.068274       1        1  22.076703
7     -0.261228  0.041523       2        1  15.209029
0      0.050265 -0.252501       3        1  14.190945
19    -0.088790 -0.233617       4        1   9.780013
6      0.147652  0.235874       5        1   8.017237
15    -0.026110  0.251963       6        1   6.490667
11    -0.161699  0.163726       7        1   6.169471
17    -0.112040 -0.140392       8        1   4.631223
18     0.206278 -0.117508       9        1   4.430601
1      0.193499  0.012375      10        1   2.882469
3      0.093123 -0.094410      11        1   2.271774
14     0.106345  0.078912      12        1   1.862960
9     -0.044736  0.043954      13        1   1.043010
4      0.022683  0.020939      14        1   0.261809
10     0.014901  0.009575      15        1   0.116509
5      0.014915  0.009572      16        1   0.113116
8      0.014915  0.009572      17        1   0.113116
2      0.014915  0.009572      18        1   0.113116
13     0.014915  0.009572      19        1   0.113116
16     0.014915  0.009572      20        1   0.113116, topic_info=        Term       Freq      Total Category  logprob  loglift
2      think  13.000000  13.000000  Default  30.0000  30.0000
26      form   5.000000   5.000000  Default  29.0000  29.0000
8       know  15.000000  15.000000  Default  28.0000  28.0000
68        go   6.000000   6.000000  Default  27.0000  27.0000
92      bill   5.000000   5.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25  download   0.002416   2.171150  Topic20  -5.4293  -0.0163
26      form   0.002416   5.458885  Topic20  -5.4293  -0.9383
27      high   0.002416   3.766665  Topic20  -5.4293  -0.5673
28   instead   0.002416   1.313872  Topic20  -5.4293   0.4859
29     issue   0.002416   2.891270  Topic20  -5.4293  -0.3028

[1016 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
97        9  0.908716    accompany
42        1  0.735491  accordingly
140       1  0.453314      account
140       3  0.453314      account
213       4  0.970118     accurate
...     ...       ...          ...
84        4  0.221807         well
84        9  0.221807         well
198       5  0.812062         word
199       5  0.996890         work
3        13  0.929318         year

[356 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 8, 1, 20, 7, 16, 12, 18, 19, 2, 4, 15, 10, 5, 11, 6, 9, 3, 14, 17])

In [180]:
pyLDAvis.save_html(vis, 'PPT05.html')